Berner Fachhochschule BFH - MAS Data Science - Graph Machine Learning - Master Thesis FS/2022 Thomas Iten

# 11. Embedding based Graph Link Prediction

**Referenzen**<br />
[1] https://snap.stanford.edu/node2vec<br />
[2] https://stellargraph.readthedocs.io/en/stable/demos/link-prediction/node2vec-link-prediction.html<br />
[3] Stamile, Marzullo, Deusebio. Graph Machine Learning. Packt Publishing 2021. ISBN 978-1-80020-449-2


In [2]:
import pandas as pd
from stellargraph.data import EdgeSplitter
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import metrics
from gml.graph.graph_builder import GraphBuilder
from gml.graph.graph_embedding import EdgeEmbedding

## 11.1 Cora Daten laden

Der Cora-Datensatz besteht aus 2708 wissenschaftlichen Veröffentlichungen, die einer von sieben Klassen zugeordnet sind. Das Zitationsnetzwerk besteht aus 5278 Links. Jede Veröffentlichung im Datensatz wird durch einen Wortvektor mit 0/1-Werten beschrieben, der das Fehlen/Vorhandensein des entsprechenden Wortes aus dem Wörterbuch angibt. Das Wörterbuch besteht aus 1433 eindeutigen Wörtern.

Die Datei cora.cites enthält die Liste aller Verbindungen, mit jeweils einem Knotenpaar pro Zeile:
```
35    1033
3229  118559
...
```
Die Liste wird mit Hilfe der GraphBuilder Klasse geladen. Die Builder Klasse erstellt neben der Edge List automatisch die Liste aller Knoten. Anschliessend wird mit der create() Methode der Graph erstellt und order und size ausgegeben.



In [9]:
cora_edges_file = "../data/cora/cora.cites"

graph = GraphBuilder().append_edges_file(cora_edges_file, sep="\t").create(detect_multi_graph=False)
props = graph.properties()

print("Graph order and size:")
print("Order =", props['order'], "(number of nodes)")
print("Size  =", props['size'], "(number of edges)")

Graph order and size:
Order = 2708 (number of nodes)
Size  = 5278 (number of edges)


## 10.2 Embeddings erstellen
Erstellung der Embeddings (Feature Vektor X) mit Node2Vec und Hadamard Embedder. Die Aufrufe sind in der Klasse Edge Embedding gekapselt. Es werden 128 Embedding Dimensionen erstellt. Für die anderen Parameter des Node2Vec Algorithmus werden die Default Werte übernommen.

In [11]:
embeddings =  EdgeEmbedding(graph.graph, dimensions=128).embeddings

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:09<00:00,  1.01it/s]


In [17]:
print("Embedding shape:")
print("Nodes    =", str(embeddings.kv.vectors.shape[0]), "(number of nodes)")
print("Features =", embeddings.kv.vectors.shape[1], "(number of features per node)")

Embedding shape:
Nodes    = 2708 (number of nodes)
Features = 128 (number of features per node)


## 11.3 Test- und Trainigsdaten aufbereiten
Für die Aufteilung der Daten wird die Klasse EdgeSplitter aus dem stellargraph.data Package verwendent.

**Testdaten**

Als erstes werden aus dem gesammten Graphen einen Teilgraphen sowie eine Liste von Verbindungen und Labels für die Tests generiert.
- test_graph   : Subset von original Graphen mit allen Knoten aber nur einem Teil der Verbindungen
- test_samples : Liste mit Knotenpaaren mit realen Verbindungen (positive edges) und Knotenpaaren die keine Verbindungen haben (negative edges)
- test_labels  : Liste mit den Labeln passend zu den Knotenpaaren der Testdaten
- test_embeddings: Feature Vektor der Testdaten

In [23]:
edgeSplitter = EdgeSplitter(graph.graph)
test_graph, test_samples, test_labels  = edgeSplitter.train_test_split(p=0.1, method="global")
test_embeddings = [embeddings[str(x[0]),str(x[1])] for x in test_samples]

** Sampled 527 positive and 527 negative edges. **


**Trainingsdaten**

Das gleich wird anschliessend für die Trainingsdaten gemacht. Bei der Initialisierung der EdgeSplitter Klassen wird zusätzlich der erstellte test_graph übergeben, so dass die Trainings- und Testdaten nicht überschneiden.

In [24]:
edgeSplitter = EdgeSplitter(test_graph, graph.graph)
train_graph, train_samples, train_labels = edgeSplitter.train_test_split(p=0.1, method="global")
train_embeddings = [embeddings[str(x[0]),str(x[1])] for x in train_samples]

** Sampled 475 positive and 475 negative edges. **


## 11.4 Training und Test mit RandomForest- und AdaBoostClassifier

**Train and Test**

In [32]:
classifiers = [RandomForestClassifier, AdaBoostClassifier]

labels = ['Classifier', 'Precision', 'Recall', 'F1-Score']
scores = []

for classifier in classifiers:
    name = classifier.__name__
    print(name, "train...")
    c = classifier(n_estimators=1000)
    c.fit(train_embeddings, train_labels)
    print(name, "test...")
    y_pred = c.predict(test_embeddings)
    score = [
        name,
        metrics.precision_score(test_labels, y_pred),
        metrics.recall_score(test_labels, y_pred),
        metrics.f1_score(test_labels, y_pred)
    ]
    scores.append(score)

RandomForestClassifier train...
RandomForestClassifier test...
AdaBoostClassifier train...
AdaBoostClassifier test...


**Scores**

Die Scores zeigen bereits gute Resultate mit den Default Werten vom Node2Vec Algorithmus und Hadamard Embedder.

In [33]:
df = df = pd.DataFrame(scores, columns=labels)
print(df)

               Classifier  Precision    Recall  F1-Score
0  RandomForestClassifier   0.982422  0.954459  0.968239
1      AdaBoostClassifier   0.961183  0.986717  0.973783


---
_The end._